In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("cleaned_SuperMarket_Analysis.csv")

df.head()


,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,sales,...,gross_margin_percentage,gross_income,rating,hour,calculated_sales,day,day_of_week,month,week,is_weekend
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,...,4.761905,26.1415,9.1,13,548.9715,5,Saturday,1,1,1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,...,4.761905,3.8200,9.6,10,80.2200,8,Friday,3,10,0
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,...,4.761905,16.2155,7.4,13,340.5255,3,Sunday,3,9,1
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,...,4.761905,23.2880,8.4,20,489.0480,27,Sunday,1,4,1
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,...,4.761905,30.2085,5.3,10,634.3785,8,Friday,2,6,0


In [3]:
daily_product_sales = (
    df.groupby(["date", "product_line"])
      .agg(
          daily_quantity=("quantity", "sum"),
          daily_sales=("sales", "sum")
      )
      .reset_index()
)
daily_product_sales.head()

,date,product_line,daily_quantity,daily_sales
0,2019-01-01,Electronic accessories,18,1382.745
1,2019-01-01,Fashion accessories,9,621.243
2,2019-01-01,Food and beverages,18,1066.023
3,2019-01-01,Health and beauty,2,132.027
4,2019-01-01,Home and lifestyle,8,399.756


In [4]:
daily_product_sales = daily_product_sales.sort_values(
    ["product_line", "date"]
)

daily_product_sales["expected_demand"] = (
    daily_product_sales
    .groupby("product_line")["daily_quantity"]
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)




In [5]:
SAFETY_STOCK = 0.15
daily_product_sales["stocked_quantity"] = (
    daily_product_sales["expected_demand"] * (1 + SAFETY_STOCK)
).round()


In [6]:
daily_product_sales["waste_quantity"] = (
    daily_product_sales["stocked_quantity"] - daily_product_sales["daily_quantity"]
)

daily_product_sales["waste_quantity"] = daily_product_sales["waste_quantity"].clip(lower=0)
daily_product_sales.head()

,date,product_line,daily_quantity,daily_sales,expected_demand,stocked_quantity,waste_quantity
0,2019-01-01,Electronic accessories,18,1382.745,18.000000,21.0,3.0
6,2019-01-02,Electronic accessories,6,138.663,12.000000,14.0,8.0
14,2019-01-04,Electronic accessories,10,705.621,11.333333,13.0,3.0
19,2019-01-05,Electronic accessories,7,690.018,10.250000,12.0,5.0
29,2019-01-07,Electronic accessories,4,326.424,9.000000,10.0,6.0


In [7]:
COST_RATIO = 0.6
daily_product_sales["unit_cost"] = (
    daily_product_sales["daily_sales"] / daily_product_sales["daily_quantity"]
) * COST_RATIO
daily_product_sales["waste_cost"] = (
    daily_product_sales["waste_quantity"] * daily_product_sales["unit_cost"]
)

In [8]:
total_waste = daily_product_sales["waste_quantity"].sum()
total_waste_cost = daily_product_sales["waste_cost"].sum()

total_waste, total_waste_cost


(1812.0, 63809.61573450791)

In [9]:
waste_by_product = (
    daily_product_sales
    .groupby("product_line")[["waste_quantity", "waste_cost"]]
    .sum()
    .sort_values("waste_cost", ascending=False)
)

waste_by_product


,waste_quantity,waste_cost
product_line,,
Sports and travel,338.0,12848.039570
Fashion accessories,303.0,12175.618565
Electronic accessories,327.0,11282.138861
Food and beverages,299.0,9603.602516
Home and lifestyle,265.0,9092.585552
Health and beauty,280.0,8807.630671


In [10]:
daily_product_sales["waste_percentage"] = (
    daily_product_sales["waste_quantity"] /
    daily_product_sales["stocked_quantity"]
) * 100


In [11]:
daily_product_sales["stocked_fixed"] = (
    daily_product_sales["expected_demand"] * 1.20
).round()

daily_product_sales["waste_fixed"] = (
    daily_product_sales["stocked_fixed"] - daily_product_sales["daily_quantity"]
).clip(lower=0)


In [12]:
comparison = daily_product_sales[[
    "waste_quantity", "waste_fixed"
]].sum()

comparison


waste_quantity    1812.0
waste_fixed       2001.0
dtype: float64

In [13]:
daily_product_sales.head()

,date,product_line,daily_quantity,daily_sales,expected_demand,stocked_quantity,waste_quantity,unit_cost,waste_cost,waste_percentage,stocked_fixed,waste_fixed
0,2019-01-01,Electronic accessories,18,1382.745,18.000000,21.0,3.0,46.09150,138.27450,14.285714,22.0,4.0
6,2019-01-02,Electronic accessories,6,138.663,12.000000,14.0,8.0,13.86630,110.93040,57.142857,14.0,8.0
14,2019-01-04,Electronic accessories,10,705.621,11.333333,13.0,3.0,42.33726,127.01178,23.076923,14.0,4.0
19,2019-01-05,Electronic accessories,7,690.018,10.250000,12.0,5.0,59.14440,295.72200,41.666667,12.0,5.0
29,2019-01-07,Electronic accessories,4,326.424,9.000000,10.0,6.0,48.96360,293.78160,60.000000,11.0,7.0
